In [0]:
%run "/Workspace/Dev/04. Common"

In [0]:
dbutils.widgets.text(name="env",defaultValue='',label='Enter the environment in lower case')
env = dbutils.widgets.get("env")


###Read Silver_Traffic Table

In [0]:
def read_silver_traffic_table(environment):
    print("Reading The Silver_Traffic Table :",end="")
    df_SilverTraffic=(spark.readStream
                             .table(f"`{environment}_catalog`.`silver`.`silver_traffic`")                             
                             )
    print(f"Reading {environment}_catalog.silver.silver_traffic Success!!")
    print("****************")
    return df_SilverTraffic


###Read Silver_Roads Table

In [0]:
def read_silver_roads_table(environment):
    print("Reading The Silver_Roads Table :",end="")
    df_SilverRoads=(spark.readStream
                             .table(f"`{environment}_catalog`.`silver`.`silver_roads`")                             
                             )
    print(f"Reading {environment}_catalog.silver.silver_roads Success!!")
    print("****************")
    return df_SilverRoads

###Creating Vehicle_Intensity Column

In [0]:
def create_VehicleIntensity(df):
    from pyspark.sql.functions import try_divide, col, coalesce, lit
    print("Creating Vehicle_Intensity Column :",end="")

    # df_veh1 = df.withColumn(
    # "invalid_link_length_flag",
    # when(
    #     col("Link_length_km").isNull() | (col("Link_length_km") == 0),
    #     lit(1)
    # ).otherwise(lit(0))
    # )

    # df_veh2=df_veh1.withColumn("Vehicle_Intensity",
    #                  col('Motor_Vehicle_Count') / col('invalid_link_length_flag')
    #                 )
    
    # df_veh=df_veh2.drop("invalid_link_length_flag"
    #                           )
    
    df_veh = df.withColumn(
    "Vehicle_Intensity",
    try_divide(
        coalesce(col("Motor_Vehicle_Count"), lit(0)),
        col("Link_length_km")
              )
                         )
                     
    print("Success!!")
    print("****************")
    return df_veh

###Creating Load_Time Column

In [0]:
def create_Load_Time(df):
    print("Creating Load_Time Column :",end="")
    from pyspark.sql.functions import current_timestamp
    df_load_time=df.withColumn("Load_Time",
                                current_timestamp()
                              )
    print("Success!!")
    print("****************")
    return df_load_time

###Write Data to Gold Traffic

In [0]:
def write_traffic_goldtable(StreamingDF,environment):
    print("Writing To Gold_Traffic Table :",end="")
    write_gold_traffic=(StreamingDF.writeStream
                            .format("delta")
                            .option("checkpointLocation",checkpoint+"GoldTrafficLoad/Checkpt/")
                            .outputMode("append")
                            .queryName("GoldTrafficWriteStream")
                            .trigger(availableNow=True)
                            .toTable(f"`{environment}_catalog`.`gold`.`gold_traffic`")
                        )
    print(f'Writing `{environment}_catalog`.`gold`.`gold_traffic` Success!')
    print("****************")                           

###Write Data to Gold Roads

In [0]:
def write_roads_goldtable(StreamingDF,environment):
    print("Writing To Gold_Roads Table :",end="")
    write_gold_roads=(StreamingDF.writeStream
                            .format("delta")
                            .option("checkpointLocation",checkpoint+"GoldRoadsLoad/Checkpt/")
                            .outputMode("append")
                            .queryName("GoldRoadsWriteStream")
                            .trigger(availableNow=True)
                            .toTable(f"`{environment}_catalog`.`gold`.`gold_roads`")
                        )
    print(f'Writing `{environment}_catalog`.`gold`.`gold_roads` Success!')
    print("****************")

## Calling all functions

In [0]:
## Reading from Silver tables
df_SilverTraffic = read_silver_traffic_table(env)
df_SilverRoads = read_silver_roads_table(env)
    
## Tranformations     
df_vehicle = create_VehicleIntensity(df_SilverTraffic)
df_FinalTraffic = create_Load_Time(df_vehicle)
df_FinalRoads = create_Load_Time(df_SilverRoads)

## Writing to gold tables    
write_traffic_goldtable(df_FinalTraffic,env)
write_roads_goldtable(df_FinalRoads,env)
